In [179]:
# Calibration with Fit
import numpy as npy
import scipy.optimize as opt
import matplotlib.pyplot as plt

class currentProfile:
    def __init__(self, label):
        self.label=label
        self.current=[]

def dataReader(filename, label):
    inputfile=open(filename,'r')
    outputData=currentProfile(label)
    currentData=[]
    while(1):
        try:
            temp=next(inputfile).split()
            if(temp[4]==str(label)):
                outputData.current.append(1e6*float(temp[2]))
        except StopIteration:
            outputData.current=npy.array(outputData.current)
            break
    return outputData

def fitFunc(time, alpha, beta, omega, gamma):
    return gamma*(1+alpha*npy.cos(2*omega*time)+beta*npy.sin(2*omega*time))

def alphaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal = (tanPsi**2 - npy.tan(P-Ps)**2) / (tanPsi**2 + npy.tan(P-Ps)**2)
    betaReal = (2 * tanPsi * cosDelta * npy.tan(P-Ps)) / \
        (tanPsi**2 + npy.tan(P-Ps)**2)
    return (alphaReal*npy.cos(2*As)-betaReal*npy.sin(2*As))/eta


def betaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal = (tanPsi**2 - npy.tan(P-Ps)**2) / (tanPsi**2 + npy.tan(P-Ps)**2)
    betaReal = (2 * tanPsi * cosDelta * npy.tan(P-Ps)) / \
        (tanPsi**2 + npy.tan(P-Ps)**2)
    return (alphaReal*npy.sin(2*As)+betaReal*npy.cos(2*As))/eta


def residualCurve(P, Ps, As, tanPsi, cosDelta, eta):
    return 1-alphaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2-betaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2


def fitCurve(P, Ps, As, tanPsi, cosDelta, eta):
    length=int(len(P)/2)
    P=P[:length]
    return npy.append(alphaCurve(P, Ps, As, tanPsi, cosDelta, eta), betaCurve(P, Ps, As, tanPsi, cosDelta, eta))


def refractionRate(tanPsi, cosDelta):
    rho = tanPsi*(cosDelta+1j*npy.sqrt(1-cosDelta**2))
    return npy.sqrt(npy.sin(63.0/180.0*npy.pi)**2*(1+(npy.tan(63.0/180.0*npy.pi))**2*((1-rho)/(1+rho))**2))


In [180]:
angles=["p"+str(5*i) for i in range(-18,19)]
length=len(angles)
measurements=[]
monitors=[]

for i in range(length):
    measurements.append(dataReader("dataQuartz/0410Quartz/measurement.txt",angles[i]))
    monitors.append(dataReader("dataQuartz/0410Quartz/monitor.txt", angles[i]))

measurementDark=dataReader("dataQuartz/0410Quartz/measurement.txt", "dark")
monitorDark=dataReader("dataQuartz/0410Quartz/monitor.txt", "dark")

In [181]:
ratios=[]
fitParameters=[]
fitParametersCOV=[]

# Data-taking 8s for single A
anglesA=npy.array([float(10*i)/180*npy.pi for i in range(1,37)])

for i in range(length):
    ratios.append((measurements[i].current[:586]-measurementDarkAverage)/monitors[i].current[:586])
    time=npy.array([0.5*j for j in range(len(ratios[i]))])
    modification=-0.14
    translate=2
    value=[]
    for j in range(len(anglesA)):
        value.append(npy.mean(ratios[i][translate+int((16+modification)*j):translate+int((16+modification)*(j+1))][2:-2]))
        # value=[ for k in range(len(time[translate+int((16+modification)*j):translate+int((16+modification)*(j+1))][2:-2]))]
    popt,pcov=opt.curve_fit(fitFunc, anglesA, value, p0=[0.86,0.353,1,1.5],maxfev=100000)
    fitParameters.append(popt)
    fitParametersCOV.append(pcov)
    fitRatio=fitFunc(anglesA,*popt)

    fig,axs=plt.subplots(2,1)
    axs[0].plot(anglesA,value,label="Data")
    axs[0].plot(anglesA,fitRatio,label="Fitted")
    axs[1].plot(anglesA,(value-fitRatio)/value,label="Residual")
    axs[1].set_xlabel("Time (s)")
    axs[0].set_ylabel("Intensity")
    axs[1].set_ylabel("Residual")
    axs[0].legend()
    axs[1].legend()
    fig.tight_layout()
    fig.savefig("figure/RatioVsAngleA"+measurements[i].label+"degreeP.jpg",dpi=500)
    plt.close()
# print(npy.mean(npy.array([fitParameters[i][2] for i in range(length)])))

In [193]:
alpha=[]
alphaError=[]
beta=[]
betaError=[]
residualError=[]
# partLength=58
# Data Average Interval 8s
for i in range(length):
    alpha.append(fitParameters[i][0])
    beta.append(fitParameters[i][1])
    alphaError.append(npy.sqrt(fitParametersCOV[i][0,0]))
    betaError.append(npy.sqrt(fitParametersCOV[i][1,1]))
    residualError.append(2*npy.sqrt(fitParametersCOV[i][0,0]*(fitParameters[i][0])**2+fitParametersCOV[i][1,1]*(fitParameters[i][1])**2))

angles=[5*i/180*npy.pi for i in range(-18,19)]
anglesFinal=npy.array(angles)
alpha=npy.array(alpha)
beta=npy.array(beta)
alphaError=npy.array(alphaError)
betaError=npy.array(betaError)
residualError=npy.array(residualError)
residual=1-alpha**2-beta**2

# (P, Ps, As, tanPsi, cosDelta, eta)
comboX=npy.append(anglesFinal,anglesFinal)
comboY=npy.append(alpha,beta)
comboSigma=npy.append(alphaError,betaError)
# comboX=npy.append(anglesFinal,npy.append(anglesFinal,anglesFinal))
# comboY=npy.append(alpha,npy.append(beta,residual))
# comboSigma=npy.append(alphaError,npy.append(betaError,residualError))
popt1,pcov1=opt.curve_fit(fitCurve, comboX, comboY, sigma=comboSigma, p0=[-0.001,-0.001, 0.199, 0.77,1.08],maxfev=1000000)
# popt2,pcov=opt.curve_fit(betaCurve, anglesFinal, beta, p0=popt1, sigma=betaError, maxfev=1000, bounds=([0,0, -1, -1, 0],[npy.pi,npy.pi, 1,1, 10]))
# bounds=([-0.1*npy.pi,-0.1*npy.pi,-0.2,-1,0.9],[0.1*npy.pi,0.1*npy.pi,0.2,1,1.3]),
# popt3,pcov=opt.curve_fit(residualCurve, anglesFinal, residual, p0=[-0.01,0.01,0,0.4,1], sigma=residualError, maxfev=1000, bounds=([-0.5*npy.pi,-0.5*npy.pi, -1, -1, 0],[0.5*npy.pi,0.5*npy.pi, 1,1, 10]))

pAngles=[-0.5*npy.pi+0.01*i for i in range(int(npy.pi/0.01))]
fig,axs=plt.subplots(3,1)
axs[0].errorbar(anglesFinal,alpha,alphaError,label="alpha")
axs[0].plot(pAngles,alphaCurve(pAngles, *popt1),label="Fit")

axs[1].errorbar(anglesFinal,beta,betaError,label="beta")
axs[1].plot(pAngles,betaCurve(pAngles, *popt1),label="Fit")

axs[2].errorbar(anglesFinal,residual,residualError,label="residual")
axs[2].plot(pAngles,residualCurve(pAngles, *popt1),label="Fit")

# axs[1].plot(angles,(ratios[i]-fitRatio)/ratios[i],label="Residual")
axs[2].set_xlabel("Angle")
axs[0].set_ylabel("alpha")
axs[1].set_ylabel("beta")
axs[2].set_ylabel("Residual")
axs[0].legend()
axs[1].legend()
axs[2].legend()
fig.tight_layout()
fig.savefig("figure/AlphaBeta.jpg",dpi=500)
plt.close()

name=["Ps", "As", "tanPsi", "cosDelta", "eta"]
for i in range(len(popt1)):
    print(str(name[i])+":"+str(popt1[i])+"±"+str(npy.sqrt(pcov1[i,i])))
print("n:"+str(refractionRate(popt1[2],popt1[3])))

Ps:0.05132800941746359±0.008355523352708139
As:0.013301402491410085±0.014614596085710583
tanPsi:0.16184087295421784±0.009783583951454558
cosDelta:0.7719634867038541±0.047463144184087105
eta:1.0854254734350435±0.02072377208659542
n:(1.5971116900956142-0.23559790918141094j)
